In [ ]:
!pip install -q -U datasets kaleido

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import defaultdict
from typing import List, Dict
import pandas as pd
import plotly.express as px
from datasets import load_dataset
import random
import numpy as np
from huggingface_hub import hf_hub_download
import os
import plotly.graph_objects as go
import json


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

#"withmartian/sql_interp_bm2_cs3_experiment_6.3"  withmartian/sql_interp_bm3_cs3_experiment_9.3"
tokenizer = AutoTokenizer.from_pretrained("withmartian/sql_interp_bm2_cs3_experiment_6.3")
model = AutoModelForCausalLM.from_pretrained("withmartian/sql_interp_bm2_cs3_experiment_6.3")

In [ ]:
github_token = 'github-'

# !pip install --upgrade git+https://{github_token}@github.com/withmartian/TinySQL.git
!pip install --upgrade git+https://{github_token}@github.com/withmartian/TinySQL.git@abir_min_circuits
import TinySQL as qts

  Cloning https://****@github.com/withmartian/TinySQL.git (to revision abir_min_circuits) to /private/var/folders/sh/7x5lnyb57z30b_6spb2t8w9m0000gn/T/pip-req-build-wu68u0wg
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/withmartian/TinySQL.git' /private/var/folders/sh/7x5lnyb57z30b_6spb2t8w9m0000gn/T/pip-req-build-wu68u0wg
  Running command git checkout -b abir_min_circuits --track origin/abir_min_circuits
  Switched to a new branch 'abir_min_circuits'
  branch 'abir_min_circuits' set up to track 'origin/abir_min_circuits'.
  Resolved https://****@github.com/withmartian/TinySQL.git to commit eac048722d0a6588b86673f6cc8164d83b330ade
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
model_num = 2                 # 0=GPT2, 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 3                       # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3
feature_name = qts.ENGTABLENAME   # ENGTABLENAME, ENGFIELDNAME, DEFTABLESTART, DEFTABLENAME, DEFFIELDNAME, DEFFIELDSEPARATOR
use_novel_names = False           # If True, we corrupt using words not found in the clean prompt or create sql e.g. "little" or "hammer"
use_synonyms_table = False
use_synonyms_field = False
batch_size = 10


# Key global "input" variables
clean_prompt = ""
corrupt_prompt = ""
clean_tokenizer_index = qts.UNKNOWN_VALUE # Tokenizer vocab index for clean word
corrupt_tokenizer_index = qts.UNKNOWN_VALUE # Tokenizer vocab index for corrupted word
answer_token_index = qts.UNKNOWN_VALUE # Token index in sql command answer of clean/corrupt word

# Key global "results" variables
clean_logit_diff = qts.UNKNOWN_VALUE
corrupt_logit_diff = qts.UNKNOWN_VALUE

In [ ]:
generator = qts.CorruptFeatureTestGenerator(model_num, cs_num, tokenizer, use_novel_names=use_novel_names, use_synonyms_field=use_synonyms_field, use_synonyms_table=use_synonyms_table, use_order_by=True)
examples = generator.generate_feature_examples(feature_name, batch_size)

# Each examples is corrupted at prompt_token_index. A resulting impact is expected at answer_token_index
prompts = []
ref_answers = []
for i, example in enumerate(examples):
    clean_tokenizer_index = example.clean_tokenizer_index
    corrupt_tokenizer_index = example.corrupt_tokenizer_index
    answer_token_index = example.answer_token_index

    # Truncate the clean_prompt at answer_token_index
    clean_prompt = example.clean_BatchItem.get_alpaca_prompt() + example.clean_BatchItem.sql_statement
    clean_tokens = tokenizer(clean_prompt)["input_ids"]
    prompts.append(clean_prompt)


In [ ]:
target_token = " duration" #" inventory" #"COUNT"
target_token_id = tokenizer.encode(target_token, add_special_tokens=False)[0]


In [ ]:
prompt = '### Instruction: show me the duration and size from the orders table ordered by duration in ascending order ### Context: CREATE TABLE orders ( duration CHAR, size TIME ) ### Response: SELECT duration, size FROM orders ORDER BY'# duration ASC'


inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = inputs["input_ids"]


In [ ]:
with torch.no_grad():
    outputs = model(input_ids=input_ids, output_hidden_states=True, return_dict=True)
    hidden_states = outputs.hidden_states

unembedding_matrix = model.lm_head.weight

In [ ]:
for layer_idx, layer_hidden in enumerate(hidden_states):
    # Choose position to probe — usually the last token of the prompt
    token_position = -1  # or -2 to go earlier
    residual = layer_hidden[0, token_position]  # shape: [hidden_dim]

    logits = torch.matmul(unembedding_matrix, residual)  # [vocab_size]

    # Logit and rank for COUNT
    count_logit = logits[target_token_id].item()
    rank = torch.argsort(logits, descending=True).tolist().index(target_token_id) + 1

    # Top predicted token
    top_token_id = torch.argmax(logits).item()
    top_token = tokenizer.decode([top_token_id])

    print(f"Layer {layer_idx}: duration logit = {count_logit:.2f}, rank = {rank}, top token = '{top_token}'")


Layer 0: duration logit = 0.01, rank = 100345, top token = ' BY'
Layer 1: duration logit = -0.01, rank = 102185, top token = '皿'
Layer 2: duration logit = 0.19, rank = 22403, top token = '皿'
Layer 3: duration logit = 0.14, rank = 51071, top token = ' locales'
Layer 4: duration logit = 0.26, rank = 37460, top token = '正宗'
Layer 5: duration logit = 0.53, rank = 14360, top token = ':';
'
Layer 6: duration logit = 0.60, rank = 12475, top token = '一族'
Layer 7: duration logit = 0.57, rank = 18494, top token = '
                        
'
Layer 8: duration logit = 0.54, rank = 11113, top token = 'izione'
Layer 9: duration logit = 0.35, rank = 30377, top token = '激活'
Layer 10: duration logit = 0.40, rank = 31730, top token = 'ordination'
Layer 11: duration logit = 0.40, rank = 40074, top token = '".



'
Layer 12: duration logit = 0.41, rank = 49843, top token = '".



'
Layer 13: duration logit = 0.33, rank = 61366, top token = '".



'
Layer 14: duration logit = 0.60, rank = 27184, top token